In [184]:
import random
import numpy as np
import pandas as pd
np.random.seed(42)

In [185]:
data1 = pd.read_csv('seeds.csv')
label1 = data1.iloc[:, -1]
data1 = data1.iloc[:, :-1]
for col in data1.columns:
    data1[col] = (data1[col] - data1[col].mean()) / data1[col].std()
datas = data1.values
labels = label1.values

In [186]:
data2 = pd.read_csv('Vowel.csv')
label2 = data2.iloc[:, -1]
data2 = data2.iloc[:, :-1]
for col in data2.columns:
    data2[col] = (data2[col] - data2[col].mean())/ data2[col].std()
dataV = data2.values

def replace_had_with_1(label2):
    mask = (label2 == 'had')
    label2[mask] = 0
    mask = (label2 == 'hed')
    label2[mask] = 1
    mask = (label2 == 'hAd')
    label2[mask] = 2
    mask = (label2 == 'hEd')
    label2[mask] = 3
    mask = (label2 == 'hid')
    label2[mask] = 4
    mask = (label2 == 'hId')
    label2[mask] = 5
    mask = (label2 == 'hod')
    label2[mask] = 6
    mask = (label2 == 'hOd')
    label2[mask] = 7
    mask = (label2 == 'hud')
    label2[mask] = 8
    mask = (label2 == 'hUd')
    label2[mask] = 9
    mask = (label2 == 'hYd')
    label2[mask] = 10

replace_had_with_1(label2)
labelV = label2.values

### Implement of k-means

In [187]:
def initial_center(initial, k):
    return initial[random.sample([i for i in range(initial.shape[0])],k)]
    
def distance(x1, x2):
    return np.linalg.norm(np.array(x1)-np.array(x2))


def k_means(initial, k, error=0.0001, stopping=100):
    initial_centers = initial_center(initial, k)
    for _ in range(stopping):
        clusters = [[] for i in range(k)]
        for point in initial:
            distances = []
            for center in initial_centers:
                distance_value = distance(point, center)
                distances.append(distance_value)
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(point)
        new_centers = []
        for cluster in clusters:
            center = np.mean(cluster, axis=0)
            new_centers.append(center)
        new_centers = np.array(new_centers)
        if np.all(np.linalg.norm(initial_centers - new_centers, axis=1) < error):
            break
        initial_centers = new_centers
    return initial_centers

### Implement of Gaussian Mixture Models

In [188]:
def multivariate_normal(x, mean, covariance):
    d = x.shape[0]
    coeff = 1 / np.sqrt((2*np.pi) * np.linalg.det(covariance))
    exp = -0.5 * np.sum((x - mean) @ np.linalg.inv(covariance) * (x - mean), axis=1)
    pdf = coeff * np.exp(exp) 
    return pdf

def gmm(data, k, max_iterations=1000):
    n, d = data.shape
    mu = np.ones(k) / k
    means = initial_center(data, k)
    covariances = []
    for _ in range(k):
        cov = np.eye(d)
        covariances.append(cov)
    def E_step(data, mu, means, covariances):
        post = np.zeros((n, k))
        for i in range(k):
            post[:, i] = mu[i] * multivariate_normal(data, means[i], covariances[i])
        post = post/ np.sum(post, axis=1, keepdims=True)
        return post
    
    def M_step(data, mu, means, covariances, post):
        Nk = np.sum(post, axis=0)
        mu = Nk / n
        means = np.dot(post.T, data) / Nk.reshape(-1,1)
        covariances = []
        for i in range(k):
            covariance = np.dot((data - means[i]).T, (data - means[i]) * post[:, i].reshape(-1,1)) / Nk[i]
            covariances.append(covariance)
        return mu, means, covariances
    
    for _ in range(max_iterations):
        post = E_step(data, mu, means, covariances)
        mu, means, covariances = M_step(data, mu, means, covariances, post)
    labels = np.argmax(post, axis=1)
    return labels, means

### Implement of Silhouette Coefficient

In [189]:
def calculate_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

def silhouette_coefficient(data, labels):
    n = data.shape[0]
    silhouette_scores = np.zeros(n)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distances[i, j] = calculate_distance(data[i], data[j])
            distances[j, i] = distances[i, j]
    for i in range(n):
        label = labels[i]
        label_indices = np.where(labels == label)[0]
        a_i = np.mean(distances[i, label_indices])
        b_distances = []
        for labelb in np.unique(labels):
            if labelb != label:
                labelb_indices = np.where(labels == labelb)[0]
                b_i = np.mean(distances[i, labelb_indices])
                b_distances.append(b_i)
        if len(b_distances)>0:
            b_i = min(b_distances)
            silhouette_scores[i] = (b_i - a_i) / max(a_i, b_i)
    return np.mean(silhouette_scores)

### Implement of Rand Index

In [190]:
def rand_index(labels_true, labels_pred):
    n = labels_true.shape[0]
    r = 0
    s = 0
    for i in range(n):
        for j in range(i+1, n):
            if labels_true[i] == labels_true[j] and labels_pred[i] == labels_pred[j]:
                r += 1
            elif labels_true[i] != labels_true[j] and labels_pred[i] != labels_pred[j]:
                s += 1
    rand_index = (r + s) / (n * (n - 1) / 2)
    return rand_index

### Implement of Normalized Mutual Information

In [191]:
def nmi(labels_true, labels_pred):
    n = labels_true.shape[0]
    clusters = []
    for label in labels_pred:
        if label not in clusters:
            clusters.append(label)
    n_clusters = len(clusters)
    classes = []
    for label in labels_true:
        if label not in classes:
            classes.append(label)
    n_classes = len(classes)
    joint = np.zeros((n_clusters, n_classes))
    labels_pred = labels_pred.astype(int)
    labels_true = labels_true.astype(int)
    for i in range(n):
        joint[clusters.index(labels_pred[i]), classes.index(labels_true[i])] += 1
    mutual = 0
    for i in range(n_clusters):
        for j in range(n_classes):
            pro_xy = joint[i, j] / n
            pro_x = np.sum(joint[i, :]) / n
            pro_y = np.sum(joint[:, j]) / n
            if pro_xy > 0:
                mutual += pro_xy * np.log(pro_xy / (pro_x * pro_y))
            elif pro_x == 0 and pro_y == 0:
                mutual += 0
    h_x = 0
    for i in range(n_clusters):
        pro_x = np.sum(joint[i, :]) / n
        if pro_x > 0:
            h_x += -pro_x * np.log(pro_x)
    h_y = 0
    for j in range(n_classes):
        pro_y = np.sum(joint[:, j]) / n
        if pro_y > 0:
            h_y += -pro_y * np.log(pro_y)
    nmi = mutual / np.sqrt(h_x * h_y)
    return nmi

In [192]:
k_values = [1,2,3]
for k in k_values:
    k_centers = k_means(datas, k)
    k_labels = []
    for data in datas:
        distances = []
        for center in k_centers:
            distance3 = np.linalg.norm(data - center)
            distances.append(distance3)
        cluster_label = np.argmin(distances)
        k_labels.append(cluster_label)
    k_labels = np.array(k_labels)
    gmm_labels, gmm_centers = gmm(datas, k)
    if k != 1:
        k_means_silhouette = silhouette_coefficient(datas, k_labels)
        gmm_silhouette = silhouette_coefficient(datas, gmm_labels)
    k_means_rand_index = rand_index(labels, k_labels)
    gmm_rand_index = rand_index(labels, gmm_labels)
    k_means_nmi = nmi(labels, k_labels)
    gmm_nmi = nmi(labels, gmm_labels)
    if k != 1:
        print("seeds: K-means Silhouette Coefficient ( k =", k,"):", k_means_silhouette)
        print("seeds: GMM Silhouette Coefficient ( k =", k,"):", gmm_silhouette)
        print()
    
    print("seeds: K-means Rand Index ( k =", k,"):", k_means_rand_index)
    print("seeds: GMM Rand Index ( k =", k,"):", gmm_rand_index)
    print()
    print("seeds: K-means NMI ( k =", k,"):", k_means_nmi)
    print("seeds: GMM NMI ( k =", k,"):", gmm_nmi)
    if k != 3:
        print("--------------------------------------------------")

C:\Users\User\AppData\Local\Temp\ipykernel_44548\1568060251.py:38: RuntimeWarning: invalid value encountered in scalar divide
  nmi = mutual / np.sqrt(h_x * h_y)


seeds: K-means Rand Index ( k = 1 ): 0.33014354066985646
seeds: GMM Rand Index ( k = 1 ): 0.33014354066985646

seeds: K-means NMI ( k = 1 ): nan
seeds: GMM NMI ( k = 1 ): nan
--------------------------------------------------
seeds: K-means Silhouette Coefficient ( k = 2 ): 0.4665442875145582
seeds: GMM Silhouette Coefficient ( k = 2 ): 0.4662520989859995

seeds: K-means Rand Index ( k = 2 ): 0.7493734335839599
seeds: GMM Rand Index ( k = 2 ): 0.7349737981316928

seeds: K-means NMI ( k = 2 ): 0.6177450842267775
seeds: GMM NMI ( k = 2 ): 0.5927561079931564
--------------------------------------------------
seeds: K-means Silhouette Coefficient ( k = 3 ): 0.4125299893883001
seeds: GMM Silhouette Coefficient ( k = 3 ): 0.32865592584127523

seeds: K-means Rand Index ( k = 3 ): 0.8995215311004785
seeds: GMM Rand Index ( k = 3 ): 0.8308954203691046

seeds: K-means NMI ( k = 3 ): 0.7307317284137268
seeds: GMM NMI ( k = 3 ): 0.6238891112443823


In [193]:
k_values = [1,2,3]
for k in k_values:
    k_centers = k_means(dataV, k)
    k_labels = []
    for data in dataV:
        distances = []
        for center in k_centers:
            distance4 = np.linalg.norm(data - center)
            distances.append(distance4)
        cluster_label = np.argmin(distances)
        k_labels.append(cluster_label)
    k_labels = np.array(k_labels)
    gmm_labels, gmm_centers = gmm(dataV, k)
    if k != 1:
        k_means_silhouette = silhouette_coefficient(dataV, k_labels)
        gmm_silhouette = silhouette_coefficient(dataV, gmm_labels)
    k_means_rand_index = rand_index(labelV, k_labels)
    gmm_rand_index = rand_index(labelV, gmm_labels)
    k_means_nmi = nmi(labelV, k_labels)
    gmm_nmi = nmi(labelV, gmm_labels)
    if k != 1:
        print("Vowel: K-means Silhouette Coefficient ( k =", k, "):", k_means_silhouette)
        print("Vowel: GMM Silhouette Coefficient ( k =", k, "):", gmm_silhouette)
        print()
    
    print("Vowel: K-means Rand Index ( k =", k, "):", k_means_rand_index)
    print("Vowel: GMM Rand Index ( k =", k, "):", gmm_rand_index)
    print()
    print("Vowel: K-means NMI ( k =", k, "):", k_means_nmi)
    print("Vowel: GMM NMI ( k =", k, "):", gmm_nmi)
    if k != 3:
        print("--------------------------------------------------")

C:\Users\User\AppData\Local\Temp\ipykernel_44548\1568060251.py:38: RuntimeWarning: invalid value encountered in scalar divide
  nmi = mutual / np.sqrt(h_x * h_y)


Vowel: K-means Rand Index ( k = 1 ): 0.08998988877654196
Vowel: GMM Rand Index ( k = 1 ): 0.08998988877654196

Vowel: K-means NMI ( k = 1 ): nan
Vowel: GMM NMI ( k = 1 ): nan
--------------------------------------------------
Vowel: K-means Silhouette Coefficient ( k = 2 ): 0.19491499982662652
Vowel: GMM Silhouette Coefficient ( k = 2 ): 0.1607413157566187

Vowel: K-means Rand Index ( k = 2 ): 0.4281153292275638
Vowel: GMM Rand Index ( k = 2 ): 0.42641786928945674

Vowel: K-means NMI ( k = 2 ): 0.037814581679525804
Vowel: GMM NMI ( k = 2 ): 0.03979281361715382
--------------------------------------------------
Vowel: K-means Silhouette Coefficient ( k = 3 ): 0.17483063949200578
Vowel: GMM Silhouette Coefficient ( k = 3 ): 0.08207008364656064

Vowel: K-means Rand Index ( k = 3 ): 0.6024736750722595
Vowel: GMM Rand Index ( k = 3 ): 0.6361920519655606

Vowel: K-means NMI ( k = 3 ): 0.10730339537694644
Vowel: GMM NMI ( k = 3 ): 0.14417222812062455


In [194]:
# from sklearn.metrics import rand_score, silhouette_score, normalized_mutual_info_score
# k_values = [2,3,4,5,6,7,8,9,10]
# for k in k_values:
#     k_centers = k_means(dataV, k)
#     k_labels = []
#     for data in dataV:
#         distances = []
#         for center in k_centers:
#             distance4 = np.linalg.norm(data - center)
#             distances.append(distance4)
#         cluster_label = np.argmin(distances)
#         k_labels.append(cluster_label)
#     k_labels = np.array(k_labels)
#     gmm_labels, gmm_centers = gmm(dataV, k)
#     print(f"RI: {rand_score(gmm_labels, labelV)}")
#     if k > 1:
#         print(f"sil: {silhouette_score(dataV, gmm_labels)}")
#         print(f"NMI: {normalized_mutual_info_score(gmm_labels, labelV)}")
    

In [195]:
# from sklearn.mixture import GaussianMixture
# k_values = [2,3,4,5,6,7,8,9,10,11]
# for k in k_values:
#     gmm = GaussianMixture(n_components=k)
#     gmm.fit(dataV)
#     labels = gmm.predict(dataV)
#     # print(f"RI: {rand_score(labels, labelV)}")
#     print(f"sil: {silhouette_score(dataV, labels)}")
#     print(f"NMI: {normalized_mutual_info_score(labels, labelV)}")